In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import numpy as np

from dataset import HealthcareDataset
from model import initialize_model

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [21]:
# Load trained model
net = initialize_model()

SAVED_MODEL_PATH = 'checkpoints/mobilenet_1'
net.load_state_dict(torch.load(SAVED_MODEL_PATH))

# Remove the classifier
net.classifier = nn.Identity()

# Freeze layers
for param in net.parameters():
    param.requires_grad = False

net.to(device)

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(9, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [8]:
# Get dataset
DATA_FOLDER = 'E:/xplore_data/data/'
HEALTH_FILE = 'data/dhs_gps.csv'
dtrain = HealthcareDataset(DATA_FOLDER, HEALTH_FILE)
dloader = torch.utils.data.DataLoader(dtrain, batch_size=1, shuffle=False, num_workers=0)

In [22]:
n = len(dtrain)
extracted_features = torch.zeros(n, 1280+1+11)
i = 0
# Iterate over data.
for x, y, z in dloader:
    x = x.to(device)
    # forward
    # track history if only in train
    with torch.set_grad_enabled(False):
        outputs = net(x)
        extracted_features[i, :1280] = torch.squeeze(outputs).cpu()
        extracted_features[i, 1280]  = y
        extracted_features[i, 1281:] = z
    i += 1

extracted_features = extracted_features.numpy()
extracted_features[:5]
        



tensor([[0.3913, 0.3916, 0.3966,  ..., 0.3200, 0.2800, 0.9600],
        [0.3740, 0.3869, 0.3651,  ..., 0.2222, 0.5714, 1.0000],
        [0.3859, 0.3692, 0.3747,  ..., 0.4091, 0.5000, 0.7778],
        [0.3847, 0.4004, 0.3848,  ..., 0.2857, 0.1429, 0.7692],
        [0.4049, 0.4063, 0.4172,  ..., 0.5556, 0.4074, 0.7647]])

array([[0.3913139 , 0.3915811 , 0.39658272, ..., 0.32      , 0.28      ,
        0.96      ],
       [0.37403837, 0.3869265 , 0.36507893, ..., 0.22222222, 0.5714286 ,
        1.        ],
       [0.38591093, 0.36918145, 0.37468004, ..., 0.4090909 , 0.5       ,
        0.7777778 ],
       ...,
       [0.3792824 , 0.38195845, 0.39226297, ..., 0.72727275, 1.        ,
        1.        ],
       [0.39773974, 0.3569862 , 0.38021162, ..., 0.5       , 0.7777778 ,
        0.8888889 ],
       [0.35612857, 0.35258484, 0.36436728, ..., 0.5       , 0.72727275,
        1.        ]], dtype=float32)